<a href="https://colab.research.google.com/github/LordLean/Extracting-Green-Bonds-Use-of-Proceeds/blob/main/green_bonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering System

In [ ]:
from IPython.display import clear_output

In [ ]:
# check which gpu we're using
!nvidia-smi

Fri Mar  4 17:25:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install simpletransformers

!pip install wget
!pip install tika

clear_output()

In [3]:
import wget
from tika import parser

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

## Training

In [9]:
# Training dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2022-05-31 12:22:50--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2022-05-31 12:22:51 (277 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



If using JSON files, the files should contain a single list of dictionaries. A dictionary represents a single context and its associated questions.
Each such dictionary contains two attributes, the "context" and "qas".

* context: The paragraph or text from which the question is asked.
* qas: A list of questions and answers.

Questions and answers are represented as dictionaries. Each dictionary in qas has the following format.

* id: (string) A unique ID for the question. Should be unique across the entire dataset.
* question: (string) A question.
* is_impossible: (bool) Indicates whether the question can be * answered correctly from the context.
* answers: (list) The list of correct answers to the question.

A single answer is represented by a dictionary with the following attributes.
*answer: (string) The answer to the question. Must be a substring of the context.
*answer_start: (int) Starting index of the answer in the context.

We can convert the SQuAD data into this format quite easily.

In [10]:
import json

with open('train-v2.0.json', 'r') as f:
    train_data = json.load(f)

train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ]

In [11]:
train_data = train_data[:5000]

In [12]:
len(train_data)

5000

In [13]:
train_args = {
      'num_train_epochs': 2,
      'max_seq_length': 128,
      'doc_stride': 64,
      'overwrite_output_dir': True,
      'reprocess_input_data': False,
      # 'train_batch_size': 2,
      # 'fp16': False,
  }

model = QuestionAnsweringModel('distilbert', 'distilbert-base-cased', args=train_args)
model.train_model(train_data)

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

Running Epoch 0 of 3:   0%|          | 0/6550 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 3:   0%|          | 0/6550 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/6550 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Scrape PDFs

In [60]:
!wget https://www.icmagroup.org/Emails/icma-vcards/amp_combined%20hydro%20projects_External%20Review%20report.pdf

--2022-05-31 13:26:09--  https://www.icmagroup.org/Emails/icma-vcards/amp_combined%20hydro%20projects_External%20Review%20report.pdf
Resolving www.icmagroup.org (www.icmagroup.org)... 91.216.93.249
Connecting to www.icmagroup.org (www.icmagroup.org)|91.216.93.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 675346 (660K) [application/pdf]
Saving to: ‘amp_combined hydro projects_External Review report.pdf.1’

amp_combined hydro  100%[===================>] 659.52K  1.01MB/s    in 0.6s    

2022-05-31 13:26:10 (1.01 MB/s) - ‘amp_combined hydro projects_External Review report.pdf.1’ saved [675346/675346]



In [62]:
filename = "https://www.icmagroup.org/Emails/icma-vcards/amp_combined%20hydro%20projects_External%20Review%20report.pdf"

parsed_pdf = parser.from_file(filename)

data = parsed_pdf['content'] 

2022-05-31 13:26:34,844 [MainThread  ] [INFO ]  Retrieving https://www.icmagroup.org/Emails/icma-vcards/amp_combined%20hydro%20projects_External%20Review%20report.pdf to /tmp/emails-icma-vcards-amp_combined-20hydro-20projects_external-20review-20report.pdf.


In [63]:
# Printing of content 
# print(data)

## Predictions

In [67]:
context_text = """
Use of Proceeds: The proceeds of the Bonds will be used to finance the Projects additional construction
costs, repay draws on a line of credit, refund a portion of AMP’s Combined Hydroelectric Projects Revenue
Bonds, Series 2009C (Tax-Exempt) (the “2009C Bonds”) previously issued to finance a portion of the 4
construction costs of the Projects, fund a deposit to the parity common reserve account, and pay the costs
of issuance of the Bonds. Please see project descriptions below for additional details.
"""

# context_text = data

In [68]:
predictions, raw_outputs = model.predict(
    [
        {
            "context": context_text,
            "qas": [
                {
                    "question": "What projects are financed?",
                    "id": "0",
                }
            ],
        }
    ]
)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9554.22it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [70]:
predictions[0]["answer"]

['additional construction costs',
 'additional construction costs, repay draws on a line of credit, refund a portion of AMP’s Combined Hydroelectric Projects Revenue Bonds, Series 2009C (Tax-Exempt) (the “2009C Bonds”) previously issued to finance a portion of the 4 construction costs of the Projects, fund a deposit to the parity common reserve account',
 '',
 'a deposit to the parity common reserve account',
 'the parity common reserve account',
 'Projects additional construction costs',
 'Projects additional construction costs, repay draws on a line of credit, refund a portion of AMP’s Combined Hydroelectric Projects Revenue Bonds, Series 2009C (Tax-Exempt) (the “2009C Bonds”) previously issued to finance a portion of the 4 construction costs of the Projects, fund a deposit to the parity common reserve account',
 'parity common reserve account',
 'additional construction costs, repay draws on a line of credit, refund a portion of AMP’s Combined Hydroelectric Projects Revenue Bonds, S